# DataBase manipulation
****

In this notebook I have been getting familiar with the the different commands to manipulate a datababase. 
<a id="top"></a>

<b>Table of contents</b><br>

1. [Using basic queries](#basicqueries)

2. [Dealing with dates and times](#dates)

3. [Object-Relational Mapping](#dates) 

2.4 [Constraints](#constraints) 

    


<a id="basicqueries"></a>

### 1.  Basic queries command List 

        1.1. AND|OR          [x]
        1.2. ALTER TABLE
        1.3. AS (alias)
        1.4. BETWEEN
        1.5. CREATE DATABASE [x]
        1.6. CREATE TABLE    [x]
        1.7. CREATE INDEX
        1.8. CREATE VIEW
        1.9. DELETE           [x]
        1.10. GRANT
        1.11. REVOKE
        1.12. COMMIT
        1.13. ROLLBACK
        1.14. SAVEPOINT
        1.15. DROP DATABASE
        1.16. DROP INDEX
        1.17. DROP TABLE       [x]
        1.18. EXISTS
        1.19. GROUP BY         [x]
        1.20. HAVING
        1.21. IN               [x]
        1.22. INSERT INTO      [x]
        1.23. INNER JOIN       [x]
        1.24. LEFT JOIN
        1.25. RIGHT JOIN
        1.26. FULL JOIN
        1.27. LIKE
        1.28. ORDER BY
        1.29. SELECT           [x]
        1.30. SELECT *         [x]
        1.31. SELECT DISTINCT
        1.32. SELECT INTO
        1.33. SELECT TOP
        1.34. TRUNCATE TABLE
        1.35. UNION
        1.36. UNION ALL
        1.37. UPDATE           [x]
        1.38. WHERE            [x]
        1.39. PRAGMA           [x]


In [1]:
# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import datetime
from datetime import timedelta
import names                                     # pip install names
from faker.providers.person.en import Provider   # pip install faker
import sqlite3

In [2]:
conn = sqlite3.connect('./fake_db.db')
c = conn.cursor()

In [ ]:
# The most basic thing is to select and view one entire table
c.execute("SELECT * FROM Hospital_features;").fetchall()

In [ ]:
# Now, select one column from a particular table
c.execute("SELECT Patient_admitted_id FROM covid_19_death;").fetchall()

In [ ]:
# Can I select more than one column?
c.execute("SELECT Patient_admitted_id,Hospital_ID FROM covid_19_death;").fetchall()

In [ ]:
# Two columns with conditions
c.execute("SELECT NHS_Number, Birthdate FROM patient_data WHERE Postcode = 'SO15 5FL' OR Postcode = 'BE01 5SA';").fetchall()


In [ ]:
# Could I insert a new patient in the patient_data table??
# Let's first count how many patient I have before and after insert a new patient

c.execute("SELECT COUNT(ID) from patient_data ;").fetchall()

In [ ]:
# I want to be one of the patients of this database
c.execute("INSERT INTO  patient_data VALUES ('00001','000001', 'Manuel Dominguez','M','1990-04-07','Spanish', 'SO15 10FL');").fetchall()

In [ ]:
# Count again, now the number must be 2001
c.execute("SELECT COUNT(ID) from patient_data ;").fetchall()

What if we make a mistake or notice an error in a record? We can use the <code>UPDATE</code> command to change existing data in a table.

In [ ]:
c.execute("UPDATE patient_data SET 'ID'= '88888' WHERE Full_Name = 'Manuel Dominguez';").fetchall()

In [ ]:
# Let's see if the new value was introduced
c.execute("select * FROM patient_data WHERE Full_name = 'Manuel Dominguez';").fetchall()

In [ ]:
# Another useful thing to be able to do is to delete records from a table.
c.execute("DELETE FROM patient_data WHERE Full_name = 'Manuel Dominguez';").fetchall()


In [ ]:
c.execute("select * FROM patient_data WHERE Full_name = 'Manuel Dominguez';").fetchall()

In [ ]:
# Le's do some group by

c.execute("SELECT Full_Name FROM patient_data GROUP BY Ethnicity;").fetchall()

In [ ]:
# Let's get the patient name and NHS number of the patients that got COVID-19.
# For this, I need to join two tables

c.execute("SELECT Full_Name, NHS_Number FROM patient_data INNER JOIN covid_19_admission ON covid_19_admission.Patient_admitted_id = patient_data.ID ;").fetchall()



In [ ]:
# Total number of patients admitted by hospitals

# First hospital:
c.execute("SELECT Hospital_ID, COUNT(*) FROM covid_19_admission GROUP BY Hospital_ID;").fetchall()[0][1]

In [ ]:
# Second hospital
c.execute("SELECT Hospital_ID, COUNT(*) FROM covid_19_admission GROUP BY Hospital_ID;").fetchall()[1][1]

In [ ]:
# Third hospital
c.execute("SELECT Hospital_ID, COUNT(*) FROM covid_19_admission GROUP BY Hospital_ID;").fetchall()[2][1]

### Notebook details
<br>
<i>Notebook created by <strong>Manuel Dominguez</strong> 

Creation date: May 2021<br>

In [66]:
import plotly.express as px              # pip install plotly


In [60]:
date2admission = pd.read_sql_query("SELECT iD, Date, Birthdate,Hospital_ID, Discharge_date FROM patient_data INNER JOIN covid_19_admission ON covid_19_admission.Patient_admitted_id = patient_data.ID ;", conn)
def from_dob_to_age(born):
    '''
    To convert DOB to age
    '''
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [77]:
options_chosen = [224323 ]

def update_age_vs_days_hospitalised_plot(options_chosen):
    
        date2admission_copy = date2admission[date2admission['Hospital_ID'].isin(options_chosen)]

        date2admission_copy['Birthdate'] = pd.to_datetime(date2admission_copy.Birthdate)
        
        date2admission_copy['Age'] = date2admission_copy['Birthdate'].apply(lambda x: from_dob_to_age(x))

        date2admission_copy['days_Hospitalised'] = (pd.to_datetime(date2admission_copy['Discharge_date']) - (pd.to_datetime(date2admission_copy['Date']))).dt.days

        date2admission_copy['trampa'] = date2admission_copy['Age'] + date2admission_copy['days_Hospitalised']
        
        fig = px.scatter(date2admission_copy, x="Age", y="trampa", title='')
   
        return fig

In [78]:
update_age_vs_days_hospitalised_plot(options_chosen)
        

In [74]:
date2admission_copy

,ID,Date,Birthdate,Hospital_ID,Discharge_date,Age,days_Hospitalised,trampa
0,79190137,2021-01-24 00:00:00,2004-09-16,224323,2021-02-24 00:00:00,16,31,47
2,97545829,2021-01-13 00:00:00,1998-09-17,224323,2021-03-02 00:00:00,22,48,70
7,85757512,2021-01-13 00:00:00,2010-10-02,224323,2021-01-24 00:00:00,10,11,21
9,35427668,2021-01-11 00:00:00,1985-07-24,224323,2021-02-28 00:00:00,35,48,83
12,86600851,2021-01-15 00:00:00,2013-06-01,224323,2021-01-27 00:00:00,7,12,19
13,22082063,2021-01-08 00:00:00,2010-10-19,224323,2021-01-15 00:00:00,10,7,17
14,12205971,2021-01-06 00:00:00,1999-11-17,224323,2021-02-01 00:00:00,21,26,47
16,88636541,2021-01-28 00:00:00,1984-11-25,224323,2021-03-14 00:00:00,36,45,81
20,7801186,2021-01-09 00:00:00,1945-08-29,224323,2021-02-21 00:00:00,75,43,118
21,70392446,2021-01-23 00:00:00,1976-06-18,224323,2021-02-27 00:00:00,44,35,79


In [71]:
fig = px.scatter(date2admission_copy, x="Age", y="trampa", title='')
fig.show()